## Chatbot

This chatbot can keep context

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002BEA42DB340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002BEA4311870>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student")])

AIMessage(content="Hello Upamanyu!\n\nIt's nice to meet you. That's great! MTech in Computer Science and Engineering is a challenging and rewarding field. What area of CSE are you specializing in?\n\nI'm happy to chat with you about anything related to technology, programming, or your studies if you'd like.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 26, 'total_tokens': 97, 'completion_time': 0.129090909, 'prompt_time': 0.002358949, 'queue_time': 0.264288941, 'total_time': 0.131449858}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--53c0d460-712a-47fe-9144-0a02be50e015-0', usage_metadata={'input_tokens': 26, 'output_tokens': 71, 'total_tokens': 97})

In [5]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student"),
    AIMessage(content="Hello Upamanyu!\n\nIt's nice to meet you. That's great! MTech in Computer Science and Engineering is a challenging and rewarding field. What area of CSE are you specializing in?\n\nI'm happy to chat with you about anything related to technology, programming, or your studies if you'd like.\n"),
    HumanMessage(content="Hey What's my name and what do I do?")
])

AIMessage(content="You are Upamanyu, an MTech CSE student!  \n\nI remember that from our first conversation. 😊  \n\nIs there anything else you'd like to talk about? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 117, 'total_tokens': 158, 'completion_time': 0.074545455, 'prompt_time': 0.007129767, 'queue_time': 0.568876352, 'total_time': 0.081675222}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--51e62f35-8c0f-4ae9-8438-7a72d1acef7f-0', usage_metadata={'input_tokens': 117, 'output_tokens': 41, 'total_tokens': 158})

### Message history

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config = {"configurable": {"session_id": "chat1"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student")],
    config=config
)

In [11]:
response.content

"Hi Upamanyu, \n\nIt's great to meet you!  MTech in CSE is a fascinating field. What area of computer science are you focusing on in your studies?  \n\nI'm eager to learn more about your interests. 😊 \n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Upamanyu.  😊  I remember! \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 181, 'total_tokens': 209, 'completion_time': 0.050909091, 'prompt_time': 0.011841106, 'queue_time': 0.253264845, 'total_time': 0.062750197}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0f777154-369c-4c70-b419-fe8d53055b25-0', usage_metadata={'input_tokens': 181, 'output_tokens': 28, 'total_tokens': 209})

#### Changing the config and seesion id

In [13]:
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

'As an AI, I have no memory of past conversations and do not know your name. Please tell me your name! 😊\n'

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey my name is George")],
    config=config1
)
response.content

"Hi George! It's nice to meet you.  \n\nWhat can I do for you today? 😄\n"

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

"Your name is George, remember?  I'm happy to help you with anything else you need.  😊\n"